## Imports

In [182]:
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram

import numpy as np
import math
import random

%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib import cm

## Dataset generation

Output of the old generator
```python
[(2,
  array([45, 11, 40, 38]),
  {(0, 2): -15, (0, 3): -9, (1, 2): -9, (1, 3): -14}),
 (2,
  array([10, 37,  9, 46]),
  {(0, 2): -20, (0, 3): -15, (1, 2): -2, (1, 3): -4}),
 (2,
  array([12, 48, 35, 38]),
  {(0, 2): -3, (0, 3): -7, (1, 2): -19, (1, 3): -8}),
 (2,
  array([ 4, 42, 28, 33]),
  {(0, 2): -11, (0, 3): -10, (1, 2): -1, (1, 3): -14}),
 (2,
  array([43, 23, 23, 18]),
  {(0, 2): -3, (0, 3): -2, (1, 2): -16, (1, 3): 0})]
  ```

In [266]:
def create_savings(n_queries, n_plans_per_query):
    savings = {}
    for i in range(n_queries-1):
        for j in range(n_plans_per_query[i]):
            s = j + np.sum(n_plans_per_query[0:i], dtype=int)
            for a in range(i+1, n_queries):
                for b in range(n_plans_per_query[a]):
                    t = b + np.sum(n_plans_per_query[:a], dtype=int)
                    savings[s, t] = random.randint(-20, 0)

    return savings

In [228]:
def create_problems(n_problems, n_queries, n_plans_per_query, cost_min = 0, cost_max = 50, savings_min = -20, savings_max = 0):
    problems = []
    for i in range(n_problems):
        problems.append((n_plans_per_query, np.random.randint(cost_min, cost_max, np.sum(n_plans_per_query)), 
            create_savings(n_queries, n_plans_per_query)))
    return problems

Problems are generated, but only work, for now, in double combinations -> no three way savings!

In [316]:
problems = create_problems(1, 3, [2,2,2])
problems

[([2, 2, 2],
  array([11, 37, 36, 13, 43, 27]),
  {(0, 2): -1,
   (0, 3): 0,
   (0, 4): -4,
   (0, 5): -18,
   (1, 2): -7,
   (1, 3): -1,
   (1, 4): -3,
   (1, 5): -5,
   (2, 4): -10,
   (2, 5): -20,
   (3, 4): -16,
   (3, 5): -5})]

We now want to classically solve the problem so we can get a ranking.

In [319]:
def solve_classical(problems):
    solutions_complete = []
    best_solution = []
    for problem in problems:
        t_cost = []
        cost = 0
        ##TO MAKE DYNAMIC YOU HAVE TO ITERATE FOR EACH QUERY
        for i in range(len(problem[1])):
            total_cost = problem[1][i]
            for j, k in [(a, b) for a, b in problem[2] if i == a]:
                total_cost += problem[1][k]
                for m, n in [(a, b) for a, b in problem[2] if k == a]:
                    total_cost += 
        solutions_complete.append(t_cost)
    return solutions_complete

IndentationError: expected an indented block (3232391884.py, line 13)

In [318]:
solution = solve_classical(problems)
print(solution)

2, 4
2, 5
3, 4
3, 5
2, 4
2, 5
3, 4
3, 5
[[-16, -9, -31, -15, -26, -41]]


We now generate the combinational bitstrings that are possible

In [186]:
n_qubits = np.sum(problems[0][0])
binary_string = []
for i, v in enumerate(problems[0][0]):
    if i == 0:
        for j in range(v):
            binary_string.append('0'*j + '1' + '0'*(v-j-1))
    else:
        copy = []
        for x in binary_string:
            for j in range(v):
                copy.append(x + '0'*j + '1' + '0'*(v-j-1))
        binary_string = copy


print(binary_string)


['1010100', '1010010', '1010001', '1001100', '1001010', '1001001', '0110100', '0110010', '0110001', '0101100', '0101010', '0101001']


Now we generate the circuits...

In [187]:
from qiskit import *
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

import numpy as np
import math

In [188]:
problems = create_problems(1, 3, [2,3,2])
problems

[([2, 3, 2],
  array([23, 47, 11, 25, 47, 43, 42]),
  {(0, 2): -19,
   (0, 3): 0,
   (0, 4): -20,
   (0, 5): -11,
   (0, 6): -8,
   (0, 7): -5,
   (1, 2): -4,
   (1, 3): -13,
   (1, 4): 0,
   (1, 5): -2,
   (1, 6): -10,
   (1, 7): -19,
   (2, 5): -6,
   (2, 6): -6,
   (3, 5): -19,
   (3, 6): -8,
   (4, 5): -14,
   (4, 6): -12})]

In [189]:
circuit = QuantumCircuit(np.sum(problems[0][0]))
circuit.h(range(circuit.width()))
for i, v in enumerate(problems[0][1]):
    circuit.ry(v, i)
circuit.barrier()

prev_i = 0
for i, v in problems[0][2]:
    if prev_i != i:
        circuit.barrier()
    circuit.crz(problems[0][2][i, v], i, v)
    prev_i = i
circuit.barrier()

circuit.draw()

CircuitError: 'Index out of range.'

In [ ]:
np.random.randint(1,4, size=5)

array([1, 3, 2, 2, 3])